In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession. \
    builder. \
    config('spark.jars', 'gs://spark-lib/bigquery/spark-bigquery-with-dependencies_2.12-0.31.0.jar'). \
    master('yarn'). \
    appName('Spark and BigQuery Integration'). \
    getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/22 23:48:58 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
23/07/22 23:48:58 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
23/07/22 23:48:58 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
23/07/22 23:48:58 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [3]:
project_id = 'dataanalytics-347914'

In [5]:
daily_product_revenue = spark. \
    read. \
    parquet('gs://airetail_mld/retail_gold.db/daily_product_revenue')

In [6]:
daily_product_revenue.printSchema()

root
 |-- order_date: string (nullable = true)
 |-- order_item_product_id: integer (nullable = true)
 |-- order_status: string (nullable = true)
 |-- product_quantity: integer (nullable = true)
 |-- product_revenue: float (nullable = true)



In [7]:
daily_product_revenue.show()

+--------------------+---------------------+---------------+----------------+---------------+
|          order_date|order_item_product_id|   order_status|product_quantity|product_revenue|
+--------------------+---------------------+---------------+----------------+---------------+
|2013-07-29 00:00:...|                  957|PENDING_PAYMENT|              21|        6299.58|
|2013-08-02 00:00:...|                 1073|       COMPLETE|              13|        2599.87|
|2013-08-02 00:00:...|                  885|         CLOSED|               7|         174.93|
|2013-08-02 00:00:...|                  282|SUSPECTED_FRAUD|               5|         159.95|
|2013-08-04 00:00:...|                 1004|         CLOSED|               3|        1199.94|
|2013-08-06 00:00:...|                  793|     PROCESSING|               5|          74.95|
|2013-08-09 00:00:...|                  282|PENDING_PAYMENT|               4|         127.96|
|2013-08-10 00:00:...|                 1073|        PENDING|

In [9]:
!gsutil ls gs://airetail_mld


If you have a compatible Python interpreter installed, you can use it by setting
the CLOUDSDK_PYTHON environment variable to point to it.

gs://airetail_mld/pythondemo/
gs://airetail_mld/retail_bronze.db/
gs://airetail_mld/retail_db/
gs://airetail_mld/retail_db_json/
gs://airetail_mld/retail_gold.db/
gs://airetail_mld/scripts/


In [10]:
spark.conf.set('temporaryGcsBucket', 'airetail_mld')

In [11]:

project_id = 'dataanalytics-347914'

In [12]:
daily_product_revenue. \
    write. \
    mode('overwrite'). \
    format('bigquery'). \
    option('table', f'{project_id}:retail.daily_product_revenue'). \
    save()

In [13]:
df = spark. \
    read. \
    format('bigquery'). \
    option('table', f'{project_id}:retail.daily_product_revenue'). \
    load()

In [14]:
df.show()

+--------------------+---------------------+------------+----------------+---------------+
|          order_date|order_item_product_id|order_status|product_quantity|product_revenue|
+--------------------+---------------------+------------+----------------+---------------+
|2013-10-27 00:00:...|                  135|      CLOSED|               1|           22.0|
|2014-01-14 00:00:...|                  135|      CLOSED|               1|           22.0|
|2014-02-03 00:00:...|                  135|      CLOSED|               1|           22.0|
|2013-11-28 00:00:...|                  135|      CLOSED|               1|           22.0|
|2013-12-26 00:00:...|                  134|      CLOSED|               1|           25.0|
|2014-02-28 00:00:...|                  134|      CLOSED|               1|           25.0|
|2013-09-11 00:00:...|                  567|      CLOSED|               1|           25.0|
|2013-11-24 00:00:...|                  567|      CLOSED|               1|           25.0|